In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-rc2'

### dataset API

https://blog.csdn.net/feixiang7701/article/details/81611356

#### dataset 的属性

dataset 可以通过element_spec查看形状和数据dtype

In [2]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(0,9))

In [3]:
dataset.element_spec

TensorSpec(shape=(), dtype=tf.int32, name=None)

In [4]:
def test_dataset_conv2d(ds,out_size,last_activation,loss):
    model = tf.keras.Sequential([
        
        tf.keras.layers.Conv2D(filters = 6,kernel_size=5,activation="sigmoid"),
        tf.keras.layers.MaxPooling2D(strides = 2),
        tf.keras.layers.Conv2D(filters = 16,kernel_size=5,activation="sigmoid"),
        tf.keras.layers.MaxPooling2D(strides = 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32,activation="sigmoid"),
        tf.keras.layers.Dense(out_size,activation=last_activation)
    ])
    
    model.compile(loss=loss,metrics=["acc"])
    
    model.fit(ds,epochs=3,steps_per_epoch=10)

#### 数据读取
可以通过from_tensor_slices,from_tensors,from_generator

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 17s 4us/step


In [8]:
dataset_1 = tf.data.Dataset.from_tensor_slices((x_train,y_train))

In [9]:
def map_fun(x,y):
    x = tf.divide(x,255)
    x = tf.expand_dims(x,axis=-1)
    return x,y

In [10]:
dataset_1 = dataset_1.map(map_func=map_fun).batch(256)

In [11]:
test_dataset_conv2d(dataset_1,10,last_activation="softmax",loss=tf.losses.sparse_categorical_crossentropy)

Train for 10 steps
Epoch 1/3
10/10 [==============================] - 3s 297ms/step - loss: 2.3466 - acc: 0.0996
Epoch 2/3
10/10 [==============================] - 1s 85ms/step - loss: 2.3044 - acc: 0.0992: 0s - loss: 2.3050 - acc: 
Epoch 3/3
10/10 [==============================] - 1s 80ms/step - loss: 2.3048 - acc: 0.0934


### 读取Numpy

In [12]:
import numpy as np

In [14]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
path = tf.keras.utils.get_file("mnist.npz",DATA_URL)

In [15]:
with np.load(path) as data:
    x_train = data["x_train"]
    y_train = data["y_train"]
    x_test = data["x_test"]
    y_test = data["y_test"]

In [17]:
dataset_2 = tf.data.Dataset.from_tensor_slices((x_train,y_train))

In [18]:
dataset_2 = dataset_2.map(map_func=map_fun)
dataset_2 = dataset_2.batch(256).shuffle(100)

In [19]:
test_dataset_conv2d(dataset_1,10,last_activation="softmax",loss=tf.losses.sparse_categorical_crossentropy)

Train for 10 steps
Epoch 1/3
10/10 [==============================] - 2s 155ms/step - loss: 2.3549 - acc: 0.1035
Epoch 2/3
10/10 [==============================] - 1s 82ms/step - loss: 2.3042 - acc: 0.1027
Epoch 3/3
10/10 [==============================] - 1s 81ms/step - loss: 2.3040 - acc: 0.1145


### Image

In [22]:
DATA_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(fname="flower_photos",origin=DATA_URL,untar=True)